# BermudanSwaption

In this notebook, we illustrate the pricing of Bermudan swaptions with DiffFusion.jl.

## Incorporating DiffFusion.jl

We setup a Julia project environment in the current directory.

In [ ]:
using Pkg
Pkg.activate(".")

We need to incorporate the DiffFusion package.

In [ ]:
using DiffFusion

And use some further packages.

In [ ]:
using Plots
# gr()
plotlyjs()

## Model Setup

For this example, we set up a one-factor Gaussian HJM model.

The model requires various inputs.

### Correlations

Correlations between risk factors are stored in a `CorrelationHolder` object.

A one-factor interest rate model does not need need correlations. But the model interface requires that input. Therefore, we set up a trivial correlation holder without correlation values.

In [ ]:
ch = DiffFusion.correlation_holder("");

### Benchmark Rates

The volatility specification of the HJM model uses *benchmark rates*. The benchmark rates are parametrised by tenors $\delta$.

For the one-factor model we choose the short rate as our benchmark rate. The corresponding tenor is zero.

In [ ]:
δ = DiffFusion.flat_parameter([ 0., ]);

### Mean Reversion Parameter

We need to specify a constant mean reversion rate per yield curve factor. For our one-factor model this is just a single value.

In [ ]:
χ = DiffFusion.flat_parameter([ 0.01, ]);

### Volatility

We use a piece-wise constant volatility function.

In [ ]:
times = [  1.,  2.,  5., 10. ]
values = [ 50.,  60.,  70.,  80., ]' * 1.0e-4 
σ = DiffFusion.backward_flat_volatility("", times, values);

### Model

In the model, we combine the various parameters.

In [ ]:
model = DiffFusion.gaussian_hjm_model("md/EUR", δ, χ, σ, ch, nothing);

## Monte Carlo Simulation

For a given model, we can now simulate state variables. The MC simulation is specified by simulation times and number of MC paths.

In this example, we use Sobol sequences as quasi-random numbers.

In [ ]:
times = 0.0:0.25:10.0
# times = 0.0:0.10:10.0
n_paths = 2^10

sim = DiffFusion.simple_simulation(
    model,
    ch,
    times,
    n_paths,
    with_progress_bar = false,
    brownian_increments = DiffFusion.sobol_brownian_increments,
);

We can inspect the simulated state variables. Data is stored in the `X` element of the simulation.

The variable `X` is a 3-dim array of size `(n_states, n_paths, n_times)`.

Number of states `n_states` are specified by the model.

In [ ]:
DiffFusion.state_alias(model)

Number of paths `n_paths` and number of simulation times `n_times` are properties of the MC simulation.

In [ ]:
size(sim.X)

For illustration, we plot the simulated paths of the $x$ variable.

In [ ]:
plot(
    times,
    sim.X[1, 1:8, :]',
    title = "Simulated Paths",
    label = nothing,
    xlabel = "simulation time",
    ylabel = "simulated value",
)

## Term Structures, Context and Path

To use a simulation for pricing we need to link market data, model, simulation and products.

This step is realised by a `Context` object and a `Path` object.

As a first step, we define curves that we need for pricing.

We keep things simple for this example and only use a discount and projection yield curve.

In [ ]:
yc_estr = DiffFusion.zero_curve(
    "yc/EUR:ESTR",
    [1.0, 3.0, 6.0, 10.0],
    [1.0, 1.0, 1.0,  1.0] .* 1e-2,
)
yc_euribor6m = DiffFusion.zero_curve(
    "yc/EUR:EURIBOR6M",
    [1.0, 3.0, 6.0, 10.0],
    [2.0, 2.0, 2.0,  2.0] .* 1e-2,
)

ts_list = [
    yc_estr,
    yc_euribor6m,
];

The `Context` links the *keys* in the products to the *aliases* of models and term structures.

In [ ]:
_empty_key = DiffFusion._empty_context_key
context = DiffFusion.Context(
    "Std",
    DiffFusion.NumeraireEntry("EUR", "md/EUR", Dict(_empty_key => "yc/EUR:ESTR")),
    Dict{String, DiffFusion.RatesEntry}([
        ("EUR", DiffFusion.RatesEntry("EUR", "md/EUR", Dict(
            _empty_key  => "yc/EUR:ESTR",
            "ESTR"      => "yc/EUR:ESTR",
            "EURIBOR6M" => "yc/EUR:EURIBOR6M",
        ))),
    ]),
    Dict{String, DiffFusion.AssetEntry}(),
    Dict{String, DiffFusion.ForwardIndexEntry}(),
    Dict{String, DiffFusion.FutureIndexEntry}(),
    Dict{String, DiffFusion.FixingEntry}(),
);

The `Path` combines the various components involves.

In [ ]:
path = DiffFusion.path(sim, ts_list, context, DiffFusion.LinearPathInterpolation);

## Product Setup

We illustrate product setup in three steps:

1. Cash flows and Vanilla swap.

2. Co-terminal European swaptions.

3. Bermudan swaption.


### Cash Flows and Vanilla Swap

We use a spot-starting 10y Vanilla swap exchanging 6m Euribor versus fixes as our example instrument.

(To keep things simple, we omit spot lag and fixing lag in this example.)

In a first step, we need to specify the fixed and floating rate cash flows.

In [ ]:
fixed_flows = [
    DiffFusion.FixedRateCoupon( 1.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 2.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 3.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 4.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 5.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 6.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 7.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 8.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon( 9.0, 0.02, 1.0),
    DiffFusion.FixedRateCoupon(10.0, 0.02, 1.0),
];

libor_flows = [
    DiffFusion.SimpleRateCoupon(0.0, 0.0, 0.5, 0.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(0.5, 0.5, 1.0, 1.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(1.0, 1.0, 1.5, 1.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(1.5, 1.5, 2.0, 2.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(2.0, 2.0, 2.5, 2.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(2.5, 2.5, 3.0, 3.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(3.0, 3.0, 3.5, 3.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(3.5, 3.5, 4.0, 4.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(4.0, 4.0, 4.5, 4.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(4.5, 4.5, 5.0, 5.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(5.0, 5.0, 5.5, 5.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(5.5, 5.5, 6.0, 6.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(6.0, 6.0, 6.5, 6.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(6.5, 6.5, 7.0, 7.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(7.0, 7.0, 7.5, 7.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(7.5, 7.5, 8.0, 8.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(8.0, 8.0, 8.5, 8.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(8.5, 8.5, 9.0, 9.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(9.0, 9.0, 9.5, 9.5, 0.5, "EUR:EURIBOR6M", nothing, nothing),
    DiffFusion.SimpleRateCoupon(9.5, 9.5, 10.0, 10.0, 0.5, "EUR:EURIBOR6M", nothing, nothing),
];

Cash flows are combined into cash flow legs.

Cash flow legs decorate cash flows with details about notional, discounting, FX conversion and payer vs. receiver.

In [ ]:
fixed_notionals = 10_000.00 * ones(length(fixed_flows))
fixed_leg = DiffFusion.cashflow_leg(
    "leg/1", fixed_flows, fixed_notionals, "EUR:ESTR", nothing,  1.0,
)

libor_notionals = 10_000.00 * ones(length(libor_flows))
libor_leg = DiffFusion.cashflow_leg(
    "leg/2", libor_flows, libor_notionals, "EUR:ESTR", nothing,  -1.0
);

A Vanilla swap is the just a collection of cash flow legs.

In [ ]:
vanilla_swap = [ fixed_leg, libor_leg ]

### European Swaptions

We consider co-terminal swaptions at 2y, 4y, 6y and 8y expiries.

To simplify setup er define a short-cut function for create the swaptions.


In [ ]:
payer_receiver = -1.0
swap_disc_curve_key = "EUR:ESTR"
settlement_type = DiffFusion.SwaptionPhysicalSettlement
notional = 10_000.00

swpt_disc_curve_key = "EUR:ESTR"
swpt_fx_key = nothing
swpt_long_short = 1.0

make_swaption(_alias, _expiry_time, _libor_coupons, _fixed_coupons, ) = DiffFusion.SwaptionLeg(
    _alias,
    _expiry_time,
    _expiry_time, # settlement_time
    _libor_coupons,
    _fixed_coupons,
    payer_receiver,
    swap_disc_curve_key,
    settlement_type,
    notional,
    swpt_disc_curve_key,
    swpt_fx_key,
    swpt_long_short,
)

In [ ]:
swaption_2y = make_swaption("leg/swpn/2y", 2.0, libor_flows[5:end], fixed_flows[3:end])
swaption_4y = make_swaption("leg/swpn/4y", 4.0, libor_flows[9:end], fixed_flows[5:end])
swaption_6y = make_swaption("leg/swpn/6y", 6.0, libor_flows[13:end], fixed_flows[7:end])
swaption_8y = make_swaption("leg/swpn/8y", 8.0, libor_flows[17:end], fixed_flows[9:end]);

### Bermudan Swaption

We consider a Bermudan option to enter into a 10y swap with first exercise in 2y (10-nc-2) and exercise every two years.

A Bermudan swaption is specified by a list of `BermudanExercise` objects.

A `BermudanExercise` object encapsulates
 - exercise time,
 - swap (i.e. legs) to enter into, and
 - method to calculate regression variables.

We use a simple co-terminal Libor rate as regression variable. Note that for a one-factor model the particular choice of regression variable is less relevant.

In [ ]:
make_regression_variables(t) = [ DiffFusion.LiborRate(t, t, 10.0, "EUR:EURIBOR6M"), ]

The swaption underlyings are forward-starting swaps.

In [ ]:
swap_2y_10y = [
    DiffFusion.cashflow_leg("leg/fixed/2y-10y",fixed_flows[3:end], fixed_notionals[3:end], "EUR:ESTR", nothing,  1.0),  # receiver
    DiffFusion.cashflow_leg("leg/libor/2y-10y",libor_flows[5:end], libor_notionals[5:end], "EUR:ESTR", nothing, -1.0),  # payer
]

swap_4y_10y = [
    DiffFusion.cashflow_leg("leg/fixed/4y-10y",fixed_flows[5:end], fixed_notionals[5:end], "EUR:ESTR", nothing,  1.0),  # receiver
    DiffFusion.cashflow_leg("leg/libor/4y-10y",libor_flows[9:end], libor_notionals[9:end], "EUR:ESTR", nothing, -1.0),  # payer
]

swap_6y_10y = [
    DiffFusion.cashflow_leg("leg/fixed/6y-10y",fixed_flows[7:end], fixed_notionals[7:end], "EUR:ESTR", nothing,  1.0),  # receiver
    DiffFusion.cashflow_leg("leg/libor/6y-10y",libor_flows[13:end], libor_notionals[13:end], "EUR:ESTR", nothing, -1.0),  # payer
]

swap_8y_10y = [
    DiffFusion.cashflow_leg("leg/fixed/6y-10y",fixed_flows[9:end], fixed_notionals[9:end], "EUR:ESTR", nothing,  1.0),  # receiver
    DiffFusion.cashflow_leg("leg/libor/6y-10y",libor_flows[17:end], libor_notionals[17:end], "EUR:ESTR", nothing, -1.0),  # payer
];

For each underlying, we setup an exercise object.

In [ ]:
exercise_2y = DiffFusion.bermudan_exercise(2.0, swap_2y_10y, make_regression_variables)
exercise_4y = DiffFusion.bermudan_exercise(4.0, swap_4y_10y, make_regression_variables)
exercise_6y = DiffFusion.bermudan_exercise(6.0, swap_6y_10y, make_regression_variables)
exercise_8y = DiffFusion.bermudan_exercise(8.0, swap_8y_10y, make_regression_variables);

Finally, we can setup the Bermudan swaption.

In [ ]:
berm = DiffFusion.bermudan_swaption_leg(
    "berm/10-nc-2",
    [ exercise_2y, exercise_4y, exercise_6y, exercise_8y, ],
    # [ exercise_2y, ],
    1.0, # long option
    "", # default discounting (curve key)
    make_regression_variables,
    nothing, # path
    nothing, # make_regression
);

### American Monte Carlo Setup

Bermudan swaption pricing requires AMC methods. AMC methods are based on regression. And regression is specified by regression method and a MC simulation for regression calibration.

In order to avoid bias in our simulation, we should use a regression calibration simulation independent from the base simulation. To simplify the setup for this example, we chose to re-use the base simulation (and corresponding `Path` object) for regression calibration.

Regression method is polynomial regression.

In [ ]:
make_regression = (C, O) -> DiffFusion.polynomial_regression(C, O, 2)
# make_regression = (C, O) -> DiffFusion.piecewise_regression(C, O, 2, [3])

We update the Bermudan swaption with the details for regression calibration.

In [ ]:
DiffFusion.reset_regression!(berm, path, make_regression)

## Scenario Calculation

Now, we have all ingredients to calculate prices of our instruments along the simulated paths of the model.

Scenario calculation is implemented in the `scenarios` function.

To get some intuition about our simulated scenario prices we plot the results along the paths. We can use the interest rate state variable of our one-factor model to illustrate the dependency of scenario prices on changes in interest rates.

In [ ]:
n_plot_paths = 2^7

make_plot(_scens) = plot(
    times,
    sim.X[1, 1:n_plot_paths, :]',
    _scens.X[1:n_plot_paths, :, 1]',
    title = "price " * _scens.leg_aliases[1],
    label = nothing,
    xlabel = "simulation time",
    ylabel = "state variable",
    zlabel = "simulated value",
    size = (800, 600),
)

### Vanilla Swap

In [ ]:
scens = DiffFusion.scenarios(vanilla_swap, times, path, "", with_progress_bar=false);
vanilla_swap_scens = DiffFusion.aggregate(scens, false, true)
vanilla_swap_scens.leg_aliases[1] = "vanilla_swap"
make_plot(vanilla_swap_scens)

We see the expected result that the simulated swap prices are (almost) linear in interest rates. Moreover, swap prices approach zero when simulation times reaches the swap maturity in 10 years.

### European Swaptions

In [ ]:
swaption_2y_scens = DiffFusion.scenarios([swaption_2y], times, path, "", with_progress_bar=false)
make_plot(swaption_2y_scens)

In [ ]:
swaption_4y_scens = DiffFusion.scenarios([swaption_4y], times, path, "", with_progress_bar=false)
make_plot(swaption_4y_scens)

In [ ]:
swaption_6y_scens = DiffFusion.scenarios([swaption_6y], times, path, "", with_progress_bar=false)
make_plot(swaption_6y_scens)

In [ ]:
swaption_8y_scens = DiffFusion.scenarios([swaption_8y], times, path, "", with_progress_bar=false)
make_plot(swaption_8y_scens)

We find that scenario prices for European swaptions are all non-negative until option expiry.

After option expiry, some paths lead to negative scenario prices. These paths correspond to realisations where the option is exercised into the swap at option expiry. Then, interest rates increase and the fixed receiver swap price becomes negative. 

### Bermudan Swaption

In [ ]:
berm_scens = DiffFusion.scenarios([berm], times, path, "", with_progress_bar=false)
make_plot(berm_scens)

The Bermudan swaption scenario prices look very similar to the European swaption scenario prices. This is expected because the majority portion of the Bermudan price is specified by the corresponding max-European price.

We observe that for some paths with very high interest rates the Bermudan option prices increase even though the option is far out of the money. This is an example of a known limitation of the AMC method and (quadratic) polynomial regression. This limitation can be mitigated by more instrument-specific regression variables or alternative regression parametrisations.

## Aggregation and XVA Calculation

Finally, we can calculate risk measures like expected exposure (EE) which are the basis for various XVA calculations.

In [ ]:
vanilla_swap_ee = DiffFusion.expected_exposure(vanilla_swap_scens)
swaption_2y_ee = DiffFusion.expected_exposure(swaption_2y_scens)
swaption_4y_ee = DiffFusion.expected_exposure(swaption_4y_scens)
swaption_6y_ee = DiffFusion.expected_exposure(swaption_6y_scens)
swaption_8y_ee = DiffFusion.expected_exposure(swaption_8y_scens)
berm_ee =  DiffFusion.expected_exposure(berm_scens);

We plot the EE for our example instruments.

In [ ]:
plot(
    times,
    [
        vanilla_swap_ee.X[1,:,1],
        swaption_2y_ee.X[1,:,1],
        swaption_4y_ee.X[1,:,1],
        swaption_6y_ee.X[1,:,1],
        swaption_8y_ee.X[1,:,1],
        berm_ee.X[1,:,1] ],
    title = "Expected Exposure",
    label = hcat(
        vanilla_swap_ee.leg_aliases[1],
        swaption_2y_ee.leg_aliases[1],
        swaption_4y_ee.leg_aliases[1],
        swaption_6y_ee.leg_aliases[1],
        swaption_8y_ee.leg_aliases[1],
        berm_ee.leg_aliases[1],
    ),
    xlabel = "simulation time",
    ylabel = "simulated value",
    size = (800, 600),
)

We see how the swaption EE relates to the EE of the Vanilla swap. The *seasonal pattern* in the graphs corresponds to the coupon cash flows of the instruments.

For the swaptions, we can also observe that prior to (first) expiry the EE is constant. This is a confirmation of the martingale property of discounted derivative prices.

Furthermore, we see that the Bermudan swaption EE is larger than the maximum European EE. This observations also confirms theoretical considerations about a positive switch option value.